# Caso de Estudio - Proyectos Finalizados

Tenemos una base de datos en Excel la cual contiene muestras de proyectos con columnas como eficiencia y esfuerzo.

Vamos a integrar los datos a análisis de afinidad, clasificación y algunas técnicas para limpieza de datos si hiciera falta.

## 1. Cargar los datos del Excel

Nota: El archivo `ProyectosFinalizados.xlsx` debe estar disponible de una forma práctica en la computadora o cargado al Colab.

Instalamos la librería `openpyxl` para poder cargar el archivo de excel

In [14]:
import numpy as np
! pip install openpyxl

In [15]:
import pandas as pd

proyectos = pd.read_excel("data/ProyectosFinalizados.xlsx", sheet_name="base")

proyectos.head()

,IDProyecto,Empresa,Peticion,Peticion.1,pet+empresa,Nombre,Cfp,Eficiencia,PDR,EsfuerzoTotalP,...,PDR S,PDR M1,PDR M2,PDR L,Lead Time,F1GestiondelaDemanda,F2Elicitacion,F3Contruccion,F4Final,Tamaño
0,1,México,8318.0,8318,8318México,Modificación a la utileria de generación de pa...,12,NaN,33.708333,404.5,...,NaN,NaN,NaN,NaN,914.0,825.0,8.0,42.0,35.0,10-29
1,2,México,9619.0,9619,9619México,Modificacion al programa Vbncarga.exe,2,NaN,147.500000,295.0,...,NaN,NaN,NaN,NaN,609.0,584.0,7.0,14.0,1.0,2-9
2,3,México,10438.0,10438,10438México,Permisos para Cambios de puntos de Sembrado- P...,11,NaN,51.375455,1019.0,...,NaN,NaN,NaN,NaN,519.0,461.0,2.0,37.0,1.0,10-29
3,4,México,10522.0,10522,10522México,Corrección del informe TabuladoCredito,11,NaN,47.727273,525.0,...,NaN,NaN,NaN,NaN,493.0,455.0,9.0,27.0,1.0,10-29
4,5,México,10528.0,10528,10528México,Corrección al Tabulado de Carteras y Hojas men...,4,NaN,26.290000,213.0,...,NaN,NaN,NaN,NaN,557.0,520.0,5.0,30.0,0.0,2-9


## 2. Reestructuramos las columnas para que sean fáciles de consumir

In [16]:
print(list(proyectos.columns.values))

['IDProyecto', 'Empresa', 'Peticion', 'Peticion.1', 'pet+empresa', 'Nombre', 'Cfp', 'Eficiencia', 'PDR', 'EsfuerzoTotalP', 'EsfuerzoTotalR', 'EsfuerzoTotalE', 'EsfuerzoTotalV', 'EsfPlaneacionP', 'EsfPlaneacionR', 'EsfPlaneacionV', 'EsfElicitacionP', 'EsfElicitacionR', 'EsfElicitacionE', 'EsfElicitacionV', 'EsfDocumentacionP', 'EsfDocumentacionR', 'EsfDocumentacionE', 'EsfDocumentacionV', 'EsfArquitecturaP', 'EsfArquitecturaR', 'EsfArquitecturaE', 'EsfArquitecturaV', 'EsfCodificacionP', 'EsfCodificacionR', 'EsfCodificacionE', 'EsfCodificacionV', 'EsfPruebasP', 'EsfPruebasR', 'EsfPruebasE', 'EsfPruebasV', 'EsfImplementacionP', 'EsfImplementacionR', 'EsfImplementacionV', 'EsfOtrosP', 'EsfOtrosR', 'EsfOtrosV', 'ArquitecturaI', 'LenguajeI', 'BDI', 'FrameworkI', 'ArquitecturaV', 'LenguajeV', 'BDV', 'FrameworkV', 'EstDuracion', 'DiasLaborales', 'DiasNaturales', 'FechaMedicion', 'Creacion', 'PorAutorizarIniciativa(Primera)', 'PorAutorizarIniciativa(Ultima)', 'PorAutorizarEvaluadorProyectos(Pri

In [17]:
def limpiar_columna(columna):
    # 1. Reemplazar caracteres especiales
    columna = columna.replace(" ", "_") \
        .replace("á", "a") \
        .replace("é", "e") \
        .replace("í", "i") \
        .replace("ó", "o") \
        .replace("ú", "u") \
        .replace("ñ", "n") \
        .replace("+", "_") \
        .replace(".", "_") \
        .replace("(", "_") \
        .replace(")", "")
    import re
    # 2. Poner un guion bajo entre cambio de letras de minúscula a mayúsculas
    columna = re.sub("([a-z])([A-Z])", "\\1_\\2", columna)
    # 3. Poner un guion bajo entre ID y letra
    columna = re.sub("ID([A-Za-z])", "ID_\\1", columna)
    # 4. Poner un guion bajo entre número y letra
    columna = re.sub("([0-9])([A-Za-z])", "\\1_\\2", columna)
    # 4. Poner un guion bajo entre letra y número
    columna = re.sub("([A-Za-z])([0-9])", "\\1_\\2", columna)
    # 5. Quitar un guion bajo entre letra y número
    columna = re.sub("([A-Za-z])_([0-9])", "\\1\\2", columna)
    # 6. Poner un guion bajo entre letra y número final
    columna = re.sub("([A-Za-z])([0-9])$", "\\1_\\2", columna)
    # 7. Convertir a mayúsculas
    columna = columna.upper()
    return columna

proyectos.columns = map(limpiar_columna, proyectos.columns.values)

proyectos.head()

,ID_PROYECTO,EMPRESA,PETICION,PETICION_1,PET_EMPRESA,NOMBRE,CFP,EFICIENCIA,PDR,ESFUERZO_TOTAL_P,...,PDR_S,PDR_M_1,PDR_M_2,PDR_L,LEAD_TIME,F1_GESTIONDELA_DEMANDA,F2_ELICITACION,F3_CONTRUCCION,F4_FINAL,TAMANO
0,1,México,8318.0,8318,8318México,Modificación a la utileria de generación de pa...,12,NaN,33.708333,404.5,...,NaN,NaN,NaN,NaN,914.0,825.0,8.0,42.0,35.0,10-29
1,2,México,9619.0,9619,9619México,Modificacion al programa Vbncarga.exe,2,NaN,147.500000,295.0,...,NaN,NaN,NaN,NaN,609.0,584.0,7.0,14.0,1.0,2-9
2,3,México,10438.0,10438,10438México,Permisos para Cambios de puntos de Sembrado- P...,11,NaN,51.375455,1019.0,...,NaN,NaN,NaN,NaN,519.0,461.0,2.0,37.0,1.0,10-29
3,4,México,10522.0,10522,10522México,Corrección del informe TabuladoCredito,11,NaN,47.727273,525.0,...,NaN,NaN,NaN,NaN,493.0,455.0,9.0,27.0,1.0,10-29
4,5,México,10528.0,10528,10528México,Corrección al Tabulado de Carteras y Hojas men...,4,NaN,26.290000,213.0,...,NaN,NaN,NaN,NaN,557.0,520.0,5.0,30.0,0.0,2-9


In [18]:
print(list(proyectos.columns.values))

['ID_PROYECTO', 'EMPRESA', 'PETICION', 'PETICION_1', 'PET_EMPRESA', 'NOMBRE', 'CFP', 'EFICIENCIA', 'PDR', 'ESFUERZO_TOTAL_P', 'ESFUERZO_TOTAL_R', 'ESFUERZO_TOTAL_E', 'ESFUERZO_TOTAL_V', 'ESF_PLANEACION_P', 'ESF_PLANEACION_R', 'ESF_PLANEACION_V', 'ESF_ELICITACION_P', 'ESF_ELICITACION_R', 'ESF_ELICITACION_E', 'ESF_ELICITACION_V', 'ESF_DOCUMENTACION_P', 'ESF_DOCUMENTACION_R', 'ESF_DOCUMENTACION_E', 'ESF_DOCUMENTACION_V', 'ESF_ARQUITECTURA_P', 'ESF_ARQUITECTURA_R', 'ESF_ARQUITECTURA_E', 'ESF_ARQUITECTURA_V', 'ESF_CODIFICACION_P', 'ESF_CODIFICACION_R', 'ESF_CODIFICACION_E', 'ESF_CODIFICACION_V', 'ESF_PRUEBAS_P', 'ESF_PRUEBAS_R', 'ESF_PRUEBAS_E', 'ESF_PRUEBAS_V', 'ESF_IMPLEMENTACION_P', 'ESF_IMPLEMENTACION_R', 'ESF_IMPLEMENTACION_V', 'ESF_OTROS_P', 'ESF_OTROS_R', 'ESF_OTROS_V', 'ARQUITECTURA_I', 'LENGUAJE_I', 'BDI', 'FRAMEWORK_I', 'ARQUITECTURA_V', 'LENGUAJE_V', 'BDV', 'FRAMEWORK_V', 'EST_DURACION', 'DIAS_LABORALES', 'DIAS_NATURALES', 'FECHA_MEDICION', 'CREACION', 'POR_AUTORIZAR_INICIATIVA_P

## 3. Inspeccionar el dataset

In [19]:
proyectos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2955 entries, 0 to 2954
Columns: 138 entries, ID_PROYECTO to TAMANO
dtypes: datetime64[ns](24), float64(42), int64(6), object(66)
memory usage: 3.1+ MB


In [20]:
proyectos.describe()

,ID_PROYECTO,PETICION,PETICION_1,CFP,EFICIENCIA,PDR,ESFUERZO_TOTAL_P,ESFUERZO_TOTAL_R,ESFUERZO_TOTAL_E,ESFUERZO_TOTAL_V,...,PDR_XS,PDR_S,PDR_M_1,PDR_M_2,PDR_L,LEAD_TIME,F1_GESTIONDELA_DEMANDA,F2_ELICITACION,F3_CONTRUCCION,F4_FINAL
count,2955.000000,2955.000000,2955.000000,2955.000000,1514.000000,2765.000000,2955.000000,2955.000000,2955.000000,2955.000000,...,552.000000,508.000000,171.000000,25.000000,2.000000,2818.000000,2812.000000,2812.000000,2811.000000,2818.000000
mean,1478.000000,11504.667783,11504.575635,44.551946,0.817883,49.728474,1251.967462,1282.559263,371.355868,755.950364,...,40.774953,28.351650,18.920621,11.668764,7.073779,310.930092,153.293385,50.853485,94.662753,18.540099
min,1.000000,21.100000,21.000000,0.000000,0.058973,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.650000,2.842121,1.449783,1.151212,6.651639,3.000000,0.000000,0.000000,0.000000,0.000000
25%,739.500000,756.100000,756.000000,5.000000,0.478629,3.083947,275.000000,220.170000,0.000000,0.000000,...,20.845625,13.236971,9.925193,6.699141,6.862709,92.000000,14.000000,7.000000,25.000000,0.000000
50%,1478.000000,12306.000000,12306.000000,15.000000,0.758016,24.315000,638.500000,586.470000,125.000000,310.580000,...,32.941973,22.493103,16.493254,9.087537,7.073779,192.000000,49.500000,15.000000,55.000000,3.000000
75%,2216.500000,19945.600000,19945.500000,46.000000,1.241265,56.857500,1465.250000,1407.280000,375.500000,908.370000,...,53.679714,36.002445,22.832040,14.656096,7.284849,403.250000,170.250000,38.000000,119.000000,14.000000
max,2955.000000,37966.100000,37966.000000,2450.000000,1.400000,2715.690000,27413.500000,48150.580000,17010.000000,18365.000000,...,131.230769,130.839737,72.365534,33.374603,7.495918,3008.000000,2192.000000,1350.000000,1365.000000,764.000000
std,853.179348,10478.873288,10478.865454,103.826405,0.405586,97.581442,1899.602525,2424.241745,848.577096,1281.481372,...,27.683001,21.231442,12.731515,8.350171,0.596995,341.690203,266.345435,110.039897,118.756275,52.195470


In [21]:
num_samples = 3 # Selecciona 3 muestras aleatorias
num_cols = 4    # Muestra de 4 en 4 columnas de las muestras

max_cols = len(proyectos.T) # Calcula el total de columnas

samples = proyectos.sample(num_samples) # Selecciona el número de muestras

# Recorre de 4 en 4 columnas
for i in range(0, max_cols, num_cols):
    # Imprime las columnas que se están visualizando
    print("Columnas del {} al {}".format(i + 1, min(i + num_cols, max_cols)))
    # Imprime los valores en el dataset de las muestras en esas columnas
    print(samples.T[i:i+num_cols].T)

Columnas del 1 al 4
     ID_PROYECTO    EMPRESA PETICION PETICION_1
2500        2501  Argentina    495.1        495
426          427  Bancoppel    101.0        101
1046        1047     México  18820.1      18820
Columnas del 5 al 8
       PET_EMPRESA                                             NOMBRE  CFP   
2500  495Argentina                  Actualización de proceso en el CT   31  \
426   101Bancoppel  101-RQI 03 523 Desglose de Importe Total en Tr...    2   
1046   18820México                MEJORAS SISTEMA RETROTRAIDO MUEBLES  181   

     EFICIENCIA  
2500   0.322096  
426         NaN  
1046        NaN  
Columnas del 9 al 12
            PDR ESFUERZO_TOTAL_P ESFUERZO_TOTAL_R ESFUERZO_TOTAL_E
2500        0.0           2194.5          1889.34              0.0
426       92.23            251.0           234.57              0.0
1046  15.386906           3252.0          1800.42           1375.0
Columnas del 13 al 16
     ESFUERZO_TOTAL_V ESF_PLANEACION_P ESF_PLANEACION_R ESF_PLANEACION_V

## 4. Explorar categorías y ejes de análisis

Definimos una función de limpieza de nombres

In [22]:
import numpy as np
import re

def limpiar_nombre(nombre):
    if type(nombre) == float and np.isnan(nombre):
        return "SIN NOMBRE"
    nombre = nombre.lower()\
                .replace("á", "a")\
                .replace("é", "e")\
                .replace("í", "i")\
                .replace("ó", "o")\
                .replace("ú", "u")\
                .replace("ñ", "n")
    nombre = re.sub("\/.*", "", nombre) # Quitar texto después de la diagonal
    nombre = re.sub("[^A-Za-z\s]*", "", nombre) # Quitar caracteres fuera de la A-Z y a-z
    nombre = re.sub("\s+", " ", nombre) # Quitar espacios repetidos
    return nombre.upper().strip() # Devuelve el nombre en mayúsculas y sin espacios finales

print("Se definió la función para limpiar nombres")

Se definió la función para limpiar nombres


Limpiamos la columna `LIDER` para su análisis

In [23]:
proyectos["LIDER"] = proyectos["LIDER"].map(limpiar_nombre)

lideres = proyectos["LIDER"].unique()
lideres.sort()

lideres

array(['ADALBERTO ZAMORA BORBOA', 'AGLAE CASTRO GARCIA', 'ALBA GUADALUPE',
       'ALBA GUADALUPE QUINONEZ SAN JUAN', 'ALEJANDRA HERNANDEZ MEZA',
       'ALEXIS VALENZUELA QUINTERO', 'ALFONSO CARRANZA BELTRAN',
       'ALFREDO MONTES SOSA', 'ALMA PATRICIA GARCIA LEYVA',
       'ALVARO LEVI MORGAN COTA', 'ANA ALICIA ACOSTA MERAZ',
       'ANDREA ITZEL ESTUDILLO ARREDONDO', 'ANDREA ROMERO MENDEZ',
       'ANDREA YAQUELIN RIVERA TORRES', 'ANGEL GABRIEL PULIDO BARRON',
       'ANGELA MARIELA CARRERA PINO', 'ANSELMO VERDUGO GASTELUM',
       'AOD JAUREGUI RUIZ', 'ARMANDO ALBERTO GUZMAN COTA',
       'ARMANDO MEDINA CARDENAS', 'ASUCENA BOJORQUEZ GUTIERREZ',
       'AURORA ROJO ZEPEDA', 'AZUCENA COTA PACHECO',
       'BEATRIZ ELENA JACOBO RAYA', 'BEATRIZ REBECA ALVARADO LOPEZ',
       'BERNARDO CARLOS BAEZ GONZALEZ', 'BETEL ANAHI LOPEZ MIRANDA',
       'BRENDA CARELI RUELAS AGUIRRE', 'BRENDA DANIELA BORBOLLA RAMIREZ',
       'CARLOS EDGAR LAFARGA TORRES', 'CARLOS GILBERTO BURGUENO GARCIA',
  

## 5. Conteos sobre ejes categóricos

Calculamos el dataset de los proyectos que ha tenido cada líder

In [52]:
proyectos["PROYECTO_COUNT"] = 1

#proyectos_lider = proyectos[ ["LIDER", "PROYECTO_COUNT"] ].groupby("LIDER").sum()
proyectos_lider = proyectos[ ["LIDER", "PROYECTO_COUNT"] ].groupby("LIDER", as_index=False).count()

proyectos_lider

,LIDER,PROYECTO_COUNT
0,ADALBERTO ZAMORA BORBOA,15
1,AGLAE CASTRO GARCIA,1
2,ALBA GUADALUPE,1
3,ALBA GUADALUPE QUINONEZ SAN JUAN,18
4,ALEJANDRA HERNANDEZ MEZA,11
...,...,...
239,YADIRA YUDITH MORALES ZAZUETA,26
240,YAGGER ALBERTO CORONA FELIX,13
241,YARELI LIZETH CALDERON CORONEL,21
242,YAZMIN DEL ROSARIO MARQUEZ BENITEZ,17


Integramos el conteo al dataset de proyectos. Esto significa que los proyectos ahora sabrán cuántos proyectos tuvo el líder para cada uno de los proyectos.

In [32]:
proyectos.sample(3)

,ID_PROYECTO,EMPRESA,PETICION,PETICION_1,PET_EMPRESA,NOMBRE,CFP,EFICIENCIA,PDR,ESFUERZO_TOTAL_P,...,PDR_M_1,PDR_M_2,PDR_L,LEAD_TIME,F1_GESTIONDELA_DEMANDA,F2_ELICITACION,F3_CONTRUCCION,F4_FINAL,TAMANO,PROYECTO_COUNT
2954,2955,Bancoppel,871.1,871,871Bancoppel,Timestamps (Complementaria 7),13,0.355932,73.074615,1089.0,...,NaN,NaN,NaN,276.0,73.0,29.0,110.0,62.0,10-29,1
116,117,México,13804.0,13804,13804México,Sorteo Regreso a Clases 2016,8,NaN,54.187500,200.0,...,NaN,NaN,NaN,29.0,4.0,3.0,22.0,0.0,2-9,1
1650,1651,Bancoppel,746.1,746,746Bancoppel,Enmascarado de datos en ventanilla y cancelaci...,45,1.400000,20.859111,962.0,...,NaN,NaN,NaN,77.0,13.0,5.0,54.0,3.0,30-99,1


In [53]:
proyectos_lider["LIDER"]

0                 ADALBERTO ZAMORA BORBOA
1                     AGLAE CASTRO GARCIA
2                          ALBA GUADALUPE
3        ALBA GUADALUPE QUINONEZ SAN JUAN
4                ALEJANDRA HERNANDEZ MEZA
                      ...                
239         YADIRA YUDITH MORALES ZAZUETA
240           YAGGER ALBERTO CORONA FELIX
241        YARELI LIZETH CALDERON CORONEL
242    YAZMIN DEL ROSARIO MARQUEZ BENITEZ
243          YOSINA MEREDITH LUGO GALAVIZ
Name: LIDER, Length: 244, dtype: object

Ejemplo para saber cuántos proyectos tiene un líder.

In [55]:
proyectos_lider[ (proyectos_lider["LIDER"] == "PAUL IVAN ONTIVEROS MARIN") ]["PROYECTO_COUNT"].values[0]

30

Calculamos los proyectos para cada líder de cada proyecto.

In [57]:
proyectos["PROYECTOS_LIDER"] = proyectos["LIDER"].map(lambda nombre: proyectos_lider[ (proyectos_lider["LIDER"] == nombre) ]["PROYECTO_COUNT"].values[0])

proyectos.sample(5)[ ["ID_PROYECTO", "EMPRESA", "LIDER", "PROYECTOS_LIDER", "EFICIENCIA"] ]

,ID_PROYECTO,EMPRESA,LIDER,PROYECTOS_LIDER,EFICIENCIA
1581,1582,Argentina,VALENTIN LOPEZ VALENZUELA,12,0.579843
137,138,México,MARIO ROMERO FUENTES,12,NaN
2049,2050,México,ADALBERTO ZAMORA BORBOA,15,1.375968
604,605,Bancoppel,RUBEN ANTONIO OJEDA MILAN,45,NaN
399,400,Bancoppel,SIN NOMBRE,204,NaN


## 6. Analizar por afinidad la siguiente hipótesis

    SI EL LIDER TIENE MÁS DE LA MEDIA DE PROYECTOS, ENTONCES LA EFICIENCIA SERÁ MAYOR A LA MEDIA DE EFICIENCIAS

Si el líder no tuviera más de la media de proyector (12 proyectos), entonces sabríamos bajo cierta confianza (por calcular), que el proyecto no será superior a la eficiencia promedio.

In [58]:
proyectos_lider["PROYECTO_COUNT"].mean()

12.110655737704919

In [59]:
dataset = proyectos[ ["ID_PROYECTO", "LIDER", "PROYECTOS_LIDER", "EFICIENCIA"] ]

dataset = dataset[ (dataset["LIDER"] != "SIN NOMBRE") ]
dataset = dataset.dropna()

dataset.sample(5)

,ID_PROYECTO,LIDER,PROYECTOS_LIDER,EFICIENCIA
2715,2716,JUAN CARLOS ANGULO GARCIA,24,0.828571
1314,1315,CRISTABEL SILVA BARRAZA,27,1.400000
2404,2405,LETICIA GUADALUPE RODRIGUEZ MARTINEZ,3,0.518248
1650,1651,IRIS MERCEDES ARIAS ZAZUETA,8,1.400000
1711,1712,JUAN CARLOS ANGULO GARCIA,24,1.400000


In [70]:
corr_proyectos_eficiencia = dataset["PROYECTOS_LIDER"].corr(dataset["EFICIENCIA"])

print("Correlación: {:.4f}".format(corr_proyectos_eficiencia))

Correlación: 0.1575


In [75]:
total = dataset[ (dataset["PROYECTOS_LIDER"] > proyectos_lider["PROYECTO_COUNT"].mean()) ].count().values[0]       # El total de proyectos donde el líder tuvo más del promedio

print("Total de proyectos donde el líder tiene más proyecto que la media: {}".format(total))

Total de proyectos donde el líder tiene más proyecto que la media: 1135


In [76]:
support = dataset[ (dataset["PROYECTOS_LIDER"] > proyectos_lider["PROYECTO_COUNT"].mean()) & (dataset["EFICIENCIA"] > dataset["EFICIENCIA"].mean()) ].count().values[0]

print("Soporte de proyectos donde el líder tiene más proyecto que la media y la eficiencia es mayor a la media: {}".format(support))

Soporte de proyectos donde el líder tiene más proyecto que la media y la eficiencia es mayor a la media: 521


In [77]:
confidence = 100 * support / total

print("Podemos afirmar con un {:.1f}% de confianza que si un líder tiene más de la media de proyectos ({:.0f}), entonces la eficiencia será mayor a la media ({:.4f})".format(confidence, proyectos_lider["PROYECTO_COUNT"].mean(), dataset["EFICIENCIA"].mean()))

Podemos afirmar con un 45.9% de confianza que si un líder tiene más de la media de proyectos (12), entonces la eficiencia será mayor a la media (0.8177)
